# Data Gathering
In this part, we will gather data from SEC site.Firstly, we will import some packages.

In [11]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime 
import time
import urllib
from urllib.request import urlopen
import pandas as pd

In [12]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}

##  Define get_13F (). 
By this function ,we can get the searched results by give some infromation , i.e. CIK, fromDate,toDate to find 13F-HR in a period.
There will be a time lag and we have to extend the time span to find the report. For example, we have to chose 2021-01-01 to 2021-2-28 to get the report of the fourth quarter of 2020.
This function will return a url of all the information of the report.

In [13]:
def get_13F(cik,fromDate,toDate):
    url_13F='https://sec.report/Document/Search/?formType=13F-HR&queryCik='+str(cik)+'&fromDate='+str(fromDate)+'&toDate='+str(toDate)+'#results'
    req_13F = urllib.request.Request(url=url_13F , headers=headers)
    html_13F=urllib.request.urlopen(req_13F).read()
    soup_13F = BeautifulSoup(html_13F, "lxml")
    for a in soup_13F.find_all('a', href=True,string=" 13F-HR 13F-HR"):
        url_of_13F=a['href']
    return url_of_13F

In [14]:
ulr_13F=get_13F('0001350694','2021-05-01','2021-05-31')
ulr_13F

'https://sec.report/Document/0001172661-21-001153/#primary_doc.xml'

## Define get_xml() function
The url gotten by get_xml() will contain so many information and we want to get the xml file of 13F-HR.
We have to find some urls and find the one which is a link of xml file.

In [25]:
def get_xml(ulr_13F):
    req_xml = urllib.request.Request(url=ulr_13F , headers=headers)
    html_xml=urllib.request.urlopen(req_xml).read()
    soup2 = BeautifulSoup(html_xml, "lxml")
    xml_url=[]
    for table in soup2.findAll('table', {'class': 'table'}):
        for td in table.findAll('td'):
            for a in td.findAll('a'):
                xml_url.append((a['href']))
    for i in range(len(xml_url)):
        if (xml_url[i].find('http')!=-1 and xml_url[i].find('.xml')!=-1 and xml_url[i].find('primary_doc')==-1 and xml_url[i].find('.html')==-1):
            final_xml_url=xml_url[i]
    return final_xml_url

In [16]:
final_url=get_xml(ulr_13F)
final_url

'https://sec.report/Document/0001172661-21-001153/infotable.xml'

## Define funtion 
We can combine the two function above to run the code in a process.

In [17]:
def xml_gather(cik,fromDate,toDate,filename):
    ulr_13F=get_13F(cik,fromDate,toDate)
    final_url=get_xml(ulr_13F)
    get_xml_file(final_url,filename)

### Define gathe file funtion
After we get the url of xml file , we can parase it and save it for the network analysis.

In [18]:
def get_xml_file(finalUrl,filename):
    req_file = urllib.request.Request(finalUrl , headers=headers)
    html_file=urllib.request.urlopen(req_file).read()
    soup_file = BeautifulSoup(html_file, "xml")
    with open(filename, "w", encoding='utf-8') as file:
        file.write(str(soup_file))
    print(filename+" has been saved") 

### Select the invest companies to research, including CIK and name.

In [19]:
cik_list=['0001067983','0000019617','0001350694','0001037389','0001179392']
fund_list=['BerkshireHathawayInc','Jpmorgan','BridgewaterAssociates','RENAISSANCE','TwoSigma']

### Define a  function to get the downloading needs in a table.

In [20]:
def get_xml_table(ciks,fund_names,fromDate,toDate,year_quarter):
    gather_xml_list={'cik':[],'fromDate':[],'toDate':[],'filename':[]}
    for i in range(len(cik_list)):
        gather_xml_list['cik'].append(cik_list[i])
        gather_xml_list['fromDate'].append(fromDate)
        gather_xml_list['toDate'].append(toDate)
        gather_xml_list['filename'].append(fund_names[i]+year_quarter+'.xml')
    return gather_xml_list

# Get files of 2021 Q1.

In [21]:

xml_file_table=pd.DataFrame(get_xml_table(cik_list,fund_list,'2021-05-01','2021-05-31','-2021q1'))
xml_file_table

,cik,fromDate,toDate,filename
0,0001067983,2021-05-01,2021-05-31,BerkshireHathawayInc-2021q1.xml
1,0000019617,2021-05-01,2021-05-31,Jpmorgan-2021q1.xml
2,0001350694,2021-05-01,2021-05-31,BridgewaterAssociates-2021q1.xml
3,0001037389,2021-05-01,2021-05-31,RENAISSANCE-2021q1.xml
4,0001179392,2021-05-01,2021-05-31,TwoSigma-2021q1.xml


In [22]:
 for i in range(xml_file_table.shape[0]):
    xml_gather(xml_file_table['cik'][i],xml_file_table['fromDate'][i],xml_file_table['toDate'][i],xml_file_table['filename'][i])

BerkshireHathawayInc-2021q1.xml has been saved
Jpmorgan-2021q1.xml has been saved
BridgewaterAssociates-2021q1.xml has been saved
RENAISSANCE-2021q1.xml has been saved
TwoSigma-2021q1.xml has been saved


# Get files of 2020 Q4.

In [23]:
xml_file_table2020q4=pd.DataFrame(get_xml_table(cik_list,fund_list,'2021-01-01','2021-02-28','-2020q4'))
xml_file_table2020q4

,cik,fromDate,toDate,filename
0,0001067983,2021-01-01,2021-02-28,BerkshireHathawayInc-2020q4.xml
1,0000019617,2021-01-01,2021-02-28,Jpmorgan-2020q4.xml
2,0001350694,2021-01-01,2021-02-28,BridgewaterAssociates-2020q4.xml
3,0001037389,2021-01-01,2021-02-28,RENAISSANCE-2020q4.xml
4,0001179392,2021-01-01,2021-02-28,TwoSigma-2020q4.xml


In [24]:
for i in range(xml_file_table2020q4.shape[0]):
    xml_gather(xml_file_table2020q4['cik'][i],xml_file_table2020q4['fromDate'][i],xml_file_table2020q4['toDate'][i],xml_file_table2020q4['filename'][i])

BerkshireHathawayInc-2020q4.xml has been saved
Jpmorgan-2020q4.xml has been saved
BridgewaterAssociates-2020q4.xml has been saved
RENAISSANCE-2020q4.xml has been saved
TwoSigma-2020q4.xml has been saved
